In [5]:
import xarray as xr
from pathlib import Path
from datetime import timedelta, date
import plotting as pl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from collections import defaultdict

In [39]:
start_time = date(1995, 3, 20)
n_days = 6

base = Path(r"C:\Users\Casper\Desktop\Processed_v5")
files = [base / pl.date_to_processed(start_time + timedelta(day)) for day in range(-n_days, n_days+1)]
raw = xr.open_mfdataset(files, concat_dim='time', combine='nested', ).load()

In [40]:
attrs = defaultdict(list)
for file in files:
    for key, value in xr.open_dataset(file).attrs.items():
        attrs[key].append(value)

In [41]:
attrs['mission_name']

['TOPEX',
 'TOPEX',
 'TOPEX',
 'TOPEX',
 ['TOPEX', 'ERS-1'],
 ['TOPEX', 'ERS-1'],
 ['TOPEX', 'ERS-1'],
 ['TOPEX', 'ERS-1'],
 'TOPEX',
 'TOPEX',
 ['TOPEX', 'ERS-1'],
 ['TOPEX', 'ERS-1'],
 ['TOPEX', 'ERS-1']]

In [42]:
vmin = -0.7
vmax = 0.7
ticksize = 20
fontsize = 20
size = 0.01
cmap = 'jet'
title = f"Input data for the gridding pipeline\n({start_time}) using {n_days} number of days"
cbar_label = "sla"


fig, ax = plt.subplots(1, 1, figsize=(13, 6), subplot_kw=dict(projection=ccrs.PlateCarree()))
im = ax.scatter(raw.lon.values, raw.lat.values, s=size, c=raw.sla.values, vmin=vmin, vmax=vmax, cmap=cmap)
cbar = plt.colorbar(im, ax=ax)
cbar.set_label(cbar_label, fontsize=fontsize)
cbar.ax.tick_params(labelsize=ticksize)

ax.coastlines()
ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
ax.set_xticks([-180,-120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax.set_yticks([-90, -60, -30, 0, 30, 60, 90], crs=ccrs.PlateCarree())
ax.set_xlabel(f"Longitude [\N{DEGREE SIGN}]", fontsize=fontsize)
ax.set_ylabel(f"Latitude [\N{DEGREE SIGN}]", fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=ticksize)
ax.tick_params(axis='both', which='minor', labelsize=ticksize)

if title is not None:
    ax.set_title(title, fontsize=fontsize)

plt.savefig(Path("Raw", f"{start_time}_nd_{n_days}.png"), format='png')
plt.close()
